# Solve the GPE in a 1D parabolic trap
### Ashton Bradley

# Introduction
In this simple example we find a ground state of the Gross-Pitaevskii equation
in a harmonic trap.

The mean field order parameter evolves according to
$$
i\hbar\frac{\partial \psi(x,t)}{\partial t}=\left(-\frac{\hbar^2\partial_x^2}{2m}+V(x,t)+g|\psi(x,t)|^2\right)\psi(x,t)
$$

# Loading the package
First, we load some useful packages.

In [ ]:
using Plots, LaTeXStrings, Pkg, Revise
gr(legend=false,titlefontsize=12,size=(500,300),transpose=true,colorbar=false);

Now load `FourierGPE`

In [ ]:
using FourierGPE

Let's define a convenient plot function

In [ ]:
function showpsi(x,ψ)
    p1 = plot(x,abs2.(ψ))
    xlabel!(L"x/a_x");ylabel!(L"|\psi|^2")
    p2 = plot(x,angle.(ψ))
    xlabel!(L"x/a_x");ylabel!(L"\textrm{phase}(\psi)")
    p = plot(p1,p2,layout=(2,1),size=(600,400))
    return p
end

# User parameters
We reserve a place for user parameters.

In [ ]:
@with_kw mutable struct Params <: UserParams @deftype Float64
    # user parameters:
    κ = 0.1
end
par = Params();

Let's set the system size, and number of spatial points

In [ ]:
L = (40.0,)
N = (512,)
μ = 25.0

Now we need to initialize the simulation object and the transforms

In [ ]:
sim = Sim(L,N,par)
@pack! sim = μ
@unpack_Sim sim;

## Declaring the potential
Let's define the trapping potential.

In [ ]:
import FourierGPE.V
V(x,t) = 0.5*x^2

We only require that it is a scalar function
because alter we will evaluate it using a broadcasted dot-call.

# Initial condition
Let's define a useful Thomas-Fermi wavefunction

In [ ]:
ψ0(x,μ,g) = sqrt(μ/g)*sqrt(max(1.0-V(x,0.0)/μ,0.0)+im*0.0)
x = X[1];

The initial state is now created as

In [ ]:
ψi = ψ0.(x,μ,g)
ϕi = kspace(ψi,sim)
@pack! sim = ϕi;

# Evolution in k-space
The `FFTW` library is used to evolve the Gross-Pitaevskii equation in k-space

In [ ]:
sol = runsim(sim);

Here we save the entire solution as a single variable `sol`.

Let's have a look at the final state and verify we have a ground state

In [ ]:
ϕg = sol[end]
ψg = xspace(ϕg,sim)
plot(x,g*abs2.(ψg),fill=(0,0.2),size=(600,200))
plot!(x,one.(x)*μ)
plot!(x,V.(x,0.0))
xlims!(-10,10); ylims!(0,1.3*μ)
title!(L"\textrm{local}\; \mu(x)")
xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")

The initial Thomas-Fermi state has been evolved for a default time $t=2/\gamma$ which is
a characteristic damping time for the dissipative system with dimensionless damping
$\gamma$. The solution will approch the ground state satisfying $L\psi_0=\mu\psi_0$ on a timescale of order
$1/\gamma$. The figure shows a smooth density profile and a completely homogeneous phase
profile over the region of finite atomic density, as required for the ground state. The indeterminate
phase evident at large $|x|$ is unimportant.

## Default simulation parameters
The default parameters are given in the declaration of `Sim`, which allows parameter interdependence.
The struct `Sim` is declared as:

In [ ]:
@with_kw mutable struct Sim{D} <: Simulation{D} @deftype Float64
    L::NTuple{D,Float64}
    N::NTuple{D,Int64}
    μ = 15.0
    g = 0.1
    γ = 0.5; @assert γ >= 0.0
    ti = 0.0
    tf = 2/γ
    Nt::Int64 = 200
    t::LinRange{Float64} = LinRange(ti,tf,Nt)
    ϕi::Array{Complex{Float64},D} = zeros(N...) |> complex
    alg::OrdinaryDiffEq.OrdinaryDiffEqAdaptiveAlgorithm = Tsit5()
    reltol::Float64 = 1e-6
    params::UserParams # optional parameters
    X::NTuple{D,Array{Float64,1}} = xvecs(L,N)
    K::NTuple{D,Array{Float64,1}} = kvecs(L,N)
    espec::Array{Complex{Float64},D} = 0.5*k2(L,N)
    flags::UInt32 = FFTW.MEASURE
    T::TransformLibrary = makeT(X,K;flags=flags)
end

where we see a set of default parameters, and then some useful transform fields built using the parameters.
Note that the transforms have to be built after building `X,K`.

# Dark soliton in harmonically trapped system
We found a ground state by imaginary time propagation.
Now we can impose a phase and density imprint consistent with a dark soliton.
We will use the solution for the homogeneous system, which will be a reasonable
approximation if we impose it on a smooth background solution.

## Imprinting a dark soliton

In [ ]:
ψf = xspace(sol[end],sim)
c = sqrt(μ)
ξ = 1/c
v = 0.5*c
xs = 0.
f = sqrt(1-(v/c)^2)

Soliton speed is determined by depth and local healing length. Start at $xs=0.0$

In [ ]:
ψs = ψf.*(f*tanh.(f*(x .-xs)/ξ).+im*v/c);
showpsi(x,ψs)
xlims!(-10,10)

## Initilize Simulation
We can recycle our earlier parameter choices, modifying the damping and simulation timescale

In [ ]:
γ = 0.0
tf = 8*pi/sqrt(2); t = LinRange(ti,tf,Nt)
dt = 0.01π/μ
simSoliton = Sim(sim;γ=γ,tf=tf,t=t)
ϕi = kspace(ψs,simSoliton)
@pack! simSoliton = ϕi
@unpack_Sim simSoliton;

In doing so, we have to specify the dimension of the simulation in this case (an improved constructor needed).

## Solve equation of motion
As before, we specify the initial condition in momentum space, and evolve

In [ ]:
sols = runsim(simSoliton);

## View the solution using Plots
Plots allows easy creation of an animated gif, as in the runnable example code below.

In [ ]:
ϕf = sols[end-4]
ψf = xspace(ϕf,simSoliton)
showpsi(x,ψf)

anim = @animate for i in 1:length(t)-4
    ψ = xspace(sols[i],simSoliton)
    y = g*abs2.(ψ)
    plot(x,y,fill=(0,0.2),size=(600,300),grid=false)
    xlims!(-10,10); ylims!(0,1.3*μ)
    title!(L"\textrm{local}\; \mu(x)")
    xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")
end

gif(anim,"./examples/soliton.gif",fps=30)

Here we just plot the final state

In [ ]:
ψ = xspace(sols[end],simSoliton)
y = g*abs2.(ψ)
plot(x,y,fill=(0,0.2),size=(600,300),grid=false)
xlims!(-10,10); ylims!(0,1.3*μ)
title!(L"\textrm{local}\; \mu(x)")
xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")